[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/ImageCollection/mapping_over_image_collection.ipynb)

**Mapping over an ImageCollection**

Reference: https://developers.google.com/earth-engine/guides/ic_mapping

To apply a function to every `Image` in an `ImageCollection` use `imageCollection.map()`. The only argument to `map()` is a function which takes one parameter: an `ee.Image`. 

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()

## Define a function

In [3]:
def minus1000(num):
    result = ee.Number(num).subtract(1000)
    return result

In [4]:
result = minus1000(2000)
print(result.getInfo())

1000


## Apply the map function

In [5]:
years = ee.List.sequence(2000, 2010)
print(years.getInfo())

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [6]:
results = years.map(minus1000)
print(results.getInfo())

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]


## Use anonymous/lambda function

In [7]:
years = ee.List.sequence(2000, 2010)

results = years.map(lambda x: ee.Number(x).subtract(1000))
print(results.getInfo())

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]


## Filter an ImageCollection

In [8]:
roi = ee.Geometry.Point(-83.92, 35.96)  # Knoxville, Tennessee

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate("2019-01-01", "2020-01-01") \
    .sort("CLOUD_COVER")

print(collection.size().getInfo())

17


In [9]:
image = collection.first()

In [10]:
# image.getInfo()

In [11]:
geemap.image_props(image).getInfo()

{'CLOUD_COVER': 0.06,
 'CLOUD_COVER_LAND': 0.06,
 'EARTH_SUN_DISTANCE': 0.994981,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 7.624,
 'GEOMETRIC_RMSE_MODEL_X': 5.066,
 'GEOMETRIC_RMSE_MODEL_Y': 5.698,
 'IMAGE_DATE': '2019-03-17',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_019035_20190317_20190325_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1553541347000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-03-17T16:11:33.5001919Z',
 'SOLAR_AZIMUTH_ANGLE': 144.944595,
 'SOLAR_ZENITH_ANGLE': 43.08416,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 19,
 'WRS_ROW': 35,
 'system:asset_size': '641.99612 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_019035_20190317',
 'system:index': 'LC08_019035_20190317',
 'system:time_end': '2019-03-17 16:11:33',
 

In [ ]:
collection.aggregate_array("CLOUD_COVER").getInfo()

In [12]:
vis_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0, 
    'max': 3000,
    'gamma': 1.0
}

Map.addLayer(image, vis_params, "First image")
Map.centerObject(image)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Select the best image each year

In [15]:
roi = ee.Geometry.Point(-83.92, 35.96)  # Knoxville, Tennessee


def best_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    
    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER") \
        .first()
    
    return image

In [16]:
start_year = 2013
end_year = 2020
years = ee.List.sequence(start_year, end_year)
year_list = years.getInfo()
print(year_list)

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [17]:
images = years.map(best_image)

In [18]:
count = images.size().getInfo()
print(count)

8


In [19]:
ee.ImageCollection(images).aggregate_array("CLOUD_COVER").getInfo()

[0.09, 0.13, 0.03, 0.01, 0.03, 0.01, 0.06, 4.61]

In [20]:
for index in range(0, count):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(year_list[index])
    Map.addLayer(image, vis_params, layer_name, False)

In [21]:
Map

Map(bottom=26027.0, center=[36.036779861716155, -83.88931834142376], controls=(WidgetControl(options=['positio…

## Set image properties

In [ ]:
collection = ee.ImageCollection(images)

In [ ]:
collection.aggregate_array("system:time_start").getInfo()

In [ ]:
collection = collection.map(lambda img: img.set({"DATE": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")}))

In [ ]:
collection.aggregate_array("DATE").getInfo()